<a href="https://colab.research.google.com/github/morarez/dataset-recommender-system/blob/main/Node2Vec_V3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Imports

In [ ]:
pip install node2vec

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Created wheel for node2vec: filename=node2vec-0.4.3-py3-none-any.whl size=5980 sha256=1c1abfdc96bf12eecf22fa5acd598f0fac96bdd1fa4dbc7c37a60a9b8f95c837
  Stored in directory: /root/.cache/pip/wheels/07/62/78/5202cb8c03cbf1593b48a8a442fca8ceec2a8c80e22318bae9
Successfully built node2vec


In [ ]:
pip install networkx==2.3

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.7 MB 28.7 MB/s 
  Created wheel for networkx: filename=networkx-2.3-py2.py3-none-any.whl size=1556008 sha256=b5edaebbbef7b23bff977d494e9f41bf30986695c045b7c68cff0b4416d793b3
  Stored in directory: /root/.cache/pip/wheels/44/e6/b8/4efaab31158e9e9ca9ed80b11f6b11130bac9a9672b3cbbeaf
Successfully built networkx
  Attempting uninstall: networkx
    Found existing installation: networkx 2.6.3
    Uninstalling networkx-2.6.3:
      Successfully uninstalled networkx-2.6.3


In [ ]:
from google.colab import drive
drive.mount("/content/gdrive", force_remount=True)
import pandas as pd
import numpy as np
import glob
import os
from sklearn.metrics.pairwise import cosine_similarity
import matplotlib.pyplot as plt
import networkx as nx
from node2vec import Node2Vec as n2v

Mounted at /content/gdrive


## Load relation dataset

In [ ]:
#relation = pd.read_json("/content/gdrive/MyDrive/Recommendation_Systems_Activities/Journal/Data/data_with_relations_v1/relations.json",lines=True)
all_files = glob.glob("/content/gdrive/MyDrive/Recommendation_Systems_Activities/Journal/Data/covid/*.json")
relation = pd.concat((pd.read_json(f,lines=True) for f in all_files))

In [ ]:
relation.drop_duplicates(inplace=True)
relation.reset_index(drop=True, inplace=True)
relation

,semantics,source,target
0,HasPart,50|doi_________::a128862e945845a6eda0055556e4c9bf,50|doi_________::a2a622df8b2ae693347feb47ffafc1ae
1,HasVersion,50|doi_dedup___::45a850328baa01f229514dcbdc2718c8,50|doi_dedup___::87f367c5a93019077b90b5ccfaa19418
2,HasVersion,50|doi_dedup___::ab446e7819e296712f207d03ad2a88d5,50|doi_dedup___::c771647c3b851a7e9d5e66eced5d1f8c
3,HasVersion,50|doi_dedup___::3193e60e43632830feb16cc7872d7022,50|doi_________::1a0d128f20c350ebd81bd9e821044bc5
4,HasPart,50|doi_________::554f8417219abc81f93e6e350869f6b3,50|doi_________::a976db31dea76600c8c6987e0e8c5f75
...,...,...,...
120128,IsSupplementedBy,50|doi_dedup___::a73cf577e214825ab16dac0935bcd7f9,50|doi_dedup___::d71a580c68cc1c064bc9682716109fe4
120129,References,50|dedup_wf_001::22ebc538f16e565cd2c1cefddbaf402e,50|doi_dedup___::5f1127f8159f413b2b31d9cd5c5684d6
120130,IsSupplementedBy,50|doi_dedup___::9ff651c1958de214abdc9ba8f0950e50,50|doi_dedup___::23814e4d445dc945fb0d12e41a168a36
120131,References,50|doi_dedup___::b25f0318046decba4829255874d3062b,50|doi_dedup___::aab13c957d53082a55bf2266fd5c4a95


In [ ]:
relation['semantics'].value_counts()

References          93167
HasPart             18639
IsSupplementedBy     4393
HasVersion           1050
Cites                1028
Documents             845
Compiles              515
IsNewVersionOf        390
IsSourceOf             88
Continues               9
IsRelatedTo             8
IsOriginalFormOf        1
Name: semantics, dtype: int64

## Create graph from relations

In [ ]:
%matplotlib inline
# Build your graph
G=nx.from_pandas_edgelist(relation, 'source', 'target')
# Plot it
#nx.draw(G, with_labels=False)
#plt.show()

In [ ]:
giant = max(nx.connected_component_subgraphs(G), key=len)

In [ ]:
nx.number_connected_components(giant)

1

In [ ]:
giant.number_of_nodes()

23666

In [ ]:
giant.number_of_edges()

92191

In [ ]:
nx.info(giant)

'Name: \nType: Graph\nNumber of nodes: 3964\nNumber of edges: 4105\nAverage degree:   2.0711'

In [ ]:
nodes_list = list(giant.nodes) #List of nodes
len(nodes_list)

23666

In [ ]:
# Save nodes to file

nodes = pd.DataFrame({'id':nodes_list}) 
nodes.to_csv('covid_nodes.csv') 


## Load artifacts dataset

In [ ]:
#df_artifacts = pd.read_json('/content/gdrive/MyDrive/Recommendation_Systems_Activities/Journal/Data/data_with_relations_v1/cleaned_concat_result.json')
all_files = glob.glob("/content/gdrive/MyDrive/Recommendation_Systems_Activities/Journal/Data/covid-final-metadata/*.json")
df_artifacts = pd.concat((pd.read_json(f,lines=True) for f in all_files))
#df_artifacts = pd.read_json('/content/gdrive/MyDrive/Recommendation_Systems_Activities/Journal/Data/Metadata_21june/metadata_v3.json')
#df_artifacts = pd.read_json('/content/gdrive/MyDrive/Recommendation_Systems_Activities/Journal/Data/ni_3900/metadata_ni_3900.json', orient='index')

In [ ]:
df_artifacts

,id,title_value,subject_value,author_name,abstract_value
0,50|doi_________::005645fa800b3a4f2fe09d2146095304,[GBIF Occurrence Download],"[GBIF, biodiversity, species occurrences]",[Occdownload Gbif.Org],A dataset containing 2724646 species occurrenc...
1,50|doi_________::23d6acc951dc4dba1946aa65d0268edf,[Occurrence Download],"[GBIF, biodiversity, species occurrences]",[Occdownload Gbif.Org],A dataset containing 1762932735 species occurr...
2,50|doi_________::62f6c0b6159c5d387224ff6d4feab667,[Occurrence Download],"[GBIF, biodiversity, species occurrences]",[Occdownload Gbif.Org],A dataset containing 3333517 species occurrenc...
3,50|doi_________::86e59e1c8e7688f780e7e83642b6915f,[GBIF Occurrence Download],"[GBIF, biodiversity, species occurrences]",[Occdownload Gbif.Org],A dataset containing 113004 species occurrence...
4,50|doi_________::e38c960a591bd46e3ce070251c72c2eb,[Occurrence Download],"[GBIF, biodiversity, species occurrences]",[Occdownload Gbif.Org],A dataset containing 2343 species occurrences ...
...,...,...,...,...,...
5434,50|doi_________::1b3fbec2ea983ea0011fee5c713b2100,[GBIF Occurrence Download],"[GBIF, biodiversity, species occurrences]",[Occdownload Gbif.Org],A dataset containing 4454 species occurrences ...
5435,50|doi_________::474b9ee02046c31dbc0745864f697aa4,[Occurrence Download],"[GBIF, biodiversity, species occurrences]",[Occdownload Gbif.Org],A dataset containing 13861960 species occurren...
5436,50|doi_________::746c3f6e360bad388b08b38b812d6eff,[Occurrence Download],"[GBIF, biodiversity, species occurrences]",[Occdownload Gbif.Org],A dataset containing 7467785 species occurrenc...
5437,50|doi_________::d9fbbc337440707a45cd1232cd7d28e0,[Occurrence Download],"[GBIF, biodiversity, species occurrences]",[Occdownload Gbif.Org],A dataset containing 1923978734 species occurr...


In [ ]:
df_artifacts['abstract_value'][4243]

'A dataset containing 3895376 species occurrences available in GBIF matching the query: { "and" : [ "Year is 2008", "Month is May", "HasCoordinate is TRUE" ] } The dataset includes 3895376 records from 2225 constituent datasets: 1 records from List of animalia, fungi and plant species recorded through naturalist observations and research activities in Benin. Data mobilized in the framework of a JRS Biodiversity Foundation project of Benin. 30 records from Flickr (01) Shieldbugs &amp; allied species. 545 records from Données de répartition de l’association FLAVIA A.D.E. acquises avec un financement public.. 7081 records from Dragonfly records from the British Dragonfly Society\'s Dragonfly Recording Network for the period up to 2014. 33 records from Canadian Museum of Nature Herbarium. 126 records from Fiocruz/COLFLEB - Coleção de Flebotomíneos. 1 records from Banco de Germoplasma del Real Jardín Botánico Juan Carlos I. 2 records from ZUEC-LEP - Coleção de Lepidoptera do Museu de Zoolog

In [ ]:
df_artifacts.drop_duplicates(subset=['id'],inplace=True)

In [ ]:
df_artifacts.reset_index(drop=True, inplace=True)

In [ ]:
df_artifacts

,id,title_value,subject_value,description_value,author_name,_c0,_c1
0,50|doi_dedup___::4c5405ce1853ba281aa310605ea4b615,[Two new remarkable genera and two new species...,"[Ecology, Evolution, Behavior and Systematics,...",[<jats:p>The Neotropical fauna of Gyponini is ...,"[Alexandre C. Domahovski, Rodney R. Cavichioli]",2632,50|doi_dedup___::4c5405ce1853ba281aa310605ea4b615
1,50|doi_________::8fa129b16b69ff1b49862aeea71fdd13,[<p><strong>Thirteen new species of Agromyzida...,"[Animal Science and Zoology, Ecology, Evolutio...",[<jats:p>We present novel rearing records of A...,"[CHARLES S. EISEMAN, OWEN LONSDALE, JOHN VAN D...",4605,50|doi_________::8fa129b16b69ff1b49862aeea71fdd13
2,50|doi_________::91fdf9bd4696aa61ebbce5cc61aa2cb6,"[Two new species of Bothriurus Peters, 1861 (S...","[Animal Science and Zoology, Ecology, Evolutio...",[<jats:p>Two new species of Bothriurid scorpio...,"[SANTOS-DA-SILVA, ANDRIA DE PAULA, CARVALHO, L...",21094,50|doi_________::91fdf9bd4696aa61ebbce5cc61aa2cb6
3,50|doi_dedup___::39a98283b52a5831f7058aeae793bb03,[The first data on species diversity of leeche...,"[Animal Science and Zoology, Ecology, Evolutio...",[The Irtysh River is one of the major waterway...,"[Kaygorodova, Irina A., Fedorova, Lyudmila I.]",11451,50|doi_dedup___::39a98283b52a5831f7058aeae793bb03
4,50|doi_dedup___::c37ba849d5caebff68328dcae92c4a30,[Review of the genus Bythonia Oman 1936 with d...,"[Animals, Hemiptera, Oman, Venezuela, Animal S...",[<jats:p>Bythonia freytagi sp. nov. is diagnos...,"[GAIANI, MARCO A.]",20769,50|doi_dedup___::c37ba849d5caebff68328dcae92c4a30
...,...,...,...,...,...,...,...
17408,50|doi_________::7f0913bf29699169567b3c730a7f71ab,[GBIF Occurrence Download],"[GBIF, biodiversity, species occurrences]",NaN,[Occdownload Gbif.Org],5282,50|doi_________::7f0913bf29699169567b3c730a7f71ab
17409,50|doi_________::c770a7d0d8b9c3b56b47d44421545e83,[GBIF Occurrence Download],"[GBIF, biodiversity, species occurrences]",NaN,[Occdownload Gbif.Org],1856,50|doi_________::c770a7d0d8b9c3b56b47d44421545e83
17410,50|doi_________::f23d0bd3c11f580f13b818cc5019da5b,[Occurrence Download],"[GBIF, biodiversity, species occurrences]",NaN,[Occdownload Gbif.Org],10338,50|doi_________::f23d0bd3c11f580f13b818cc5019da5b
17411,50|doi_________::1a272c0dd418d8d6c43e5f094950d836,[Occurrence Download],"[GBIF, biodiversity, species occurrences]",NaN,[Occdownload Gbif.Org],14297,50|doi_________::1a272c0dd418d8d6c43e5f094950d836


In [ ]:
df_artifacts['title_value'][4448]

['Description of the puparium of Hermetia pulchra (Diptera: Stratiomyidae) from Brazil.',
 'Description of the puparium of Hermetia pulchra (Diptera: Stratiomyidae) from Brazil']

In [ ]:
df_artifacts[df_artifacts['id']=='50|doi_dedup___::1c2ee6dc6a31f81b1ede4fb8f764ce5c']

,id,title_value,subject_value,description_value,author_name,_c0,_c1


In [ ]:
##### counting artifacts #####
pubs = 0
not_found = 0
for item in nodes_list:
  item_type = df_artifacts[df_artifacts['id'] == item]['title_value'].tolist()
  try:
    if item_type[0] == 'software':
      pubs = pubs + 1
  except:
    giant.remove_node(item)
    #print(item)


In [ ]:
nx.number_connected_components(giant)

11975

In [ ]:
final_giant = max(nx.connected_component_subgraphs(giant), key=len)

In [ ]:
nx.info(final_giant)

'Name: \nType: Graph\nNumber of nodes: 5439\nNumber of edges: 12644\nAverage degree:   4.6494'

In [ ]:
nodes_list_final = list(final_giant.nodes) #List of nodes
len(nodes_list_final)

5439

In [ ]:
# Save nodes to file

nodes = pd.DataFrame({'id':nodes_list_final}) 
nodes.to_csv('covid_nodes_final.csv') 

## some tests on keywords

In [ ]:
test = df_artifacts.explode('subject_value')

In [ ]:
test['subject_value'].value_counts()

Ecology, Evolution, Behavior and Systematics    4604
Animal Science and Zoology                      3946
biology.organism_classification                 3883
biology                                         3160
Zoology                                         2599
                                                ... 
Gadidae                                            1
Phycidae                                           1
Merlucciidae                                       1
Macrouridae                                        1
Gamasellodes                                       1
Name: subject_value, Length: 11636, dtype: int64

## Find missing nodes in the data (No need to run anymore)

In [ ]:
'''
nodes = pd.DataFrame(nodes_list, columns =['id'])
df_final = pd.concat([df_artifacts.set_index('id'),nodes.set_index('id')], axis=1, join='inner')
df_final['id'] = df_final.index
df_final.reset_index(drop=True, inplace=True)
df_diff = pd.concat([df_final['id'],nodes['id']]).drop_duplicates(keep=False)
df_diff.reset_index(drop=True, inplace=True)
'''

In [ ]:
# missing = df_diff.to_frame()
# missing.to_csv('missing_ni.csv')

## Remove the ones with no metadata from the graph

In [ ]:
##### counting artifacts #####
pubs = 0
not_found = 0
for item in nodes_list:
  item_type = df_artifacts[df_artifacts['id'] == item]['type'].tolist()
  try:
    if item_type[0] == 'software':
      pubs = pubs + 1
  except:
    giant.remove_node(item)
    #print(item)
pubs

77

In [ ]:
nx.number_connected_components(giant)

1

In [ ]:
nx.info(giant)

'Name: \nType: Graph\nNumber of nodes: 3906\nNumber of edges: 4043\nAverage degree:   2.0701'

## Preparing data for dataset RecSys

In [ ]:
users_d = pd.read_json('/content/gdrive/MyDrive/Recommendation_Systems_Activities/Journal/Data/ni_3900/users_ni_3900.json',orient='index')

In [ ]:
users_d = users_d.loc[users_d.astype(str).drop_duplicates(subset=['id']).index]

In [ ]:
def d_counter(user_prof):
  d_counter = 0
  for id in user_prof:
    item_type = df_artifacts[df_artifacts['id'] == id]['type']
    for i in item_type:
      if str(i) == 'dataset':
        d_counter += 1
  return d_counter

In [ ]:
users_d['d_counter'] = users_d['id'].apply(lambda row : d_counter(row))

In [ ]:
users_d = users_d[users_d['d_counter'] != 0 ]
users_d = users_d[users_d['d_counter'] != 1 ]
users_d = users_d[users_d['d_counter'] != 2 ]
users_d = users_d[users_d['d_counter'] != 3 ]

### some test on users

In [ ]:
users_d['d_counter'].value_counts()

4     62
5     27
6     23
8     10
9     10
7      8
10     4
11     3
12     3
13     1
16     1
22     1
Name: d_counter, dtype: int64

In [ ]:
test = users_d.explode('id')

In [ ]:
repetition = test['id'].value_counts()

In [ ]:
repetition.value_counts()

1    211
2    100
3     92
4     36
5     16
6     15
8      5
7      5
Name: id, dtype: int64

In [ ]:
pd.set_option('display.max_rows', None)

In [ ]:
users_d['p_counter'].value_counts()

KeyError: ignored

### Leave 1 out

In [ ]:
# ## IMPORTANT: if leave-2-out/leave-3-out has already been run, you need to load users from the start

def leave_one_out_d(profile_items):
  for item in profile_items:
      item_type = df_artifacts[df_artifacts['id'] == item]['type'].tolist()
      if item_type[0] == 'dataset':
        profile_items.remove(item)
        return item

In [ ]:
users_d['leave_one_out_d'] = users_d['id'].apply(lambda x:leave_one_out_d(x))

### Leave 2 out

In [ ]:
users_d[users_d['id'].isna()]

In [ ]:
## IMPORTANT: if leave-1-out/leave-3-out has already been run, you need to load users from the start

def leave_two_out_d(profile_items):
  dataset_list = []
  for item in profile_items:
      item_type = df_artifacts[df_artifacts['id'] == item]['type'].tolist()
      if item_type[0] == 'dataset':
          dataset_list.append(item)
          if len(dataset_list) == 2:
            for ds in dataset_list:
              profile_items.remove(ds)
            return dataset_list

In [ ]:
users_d['leave_two_out_d'] = users_d['id'].apply(lambda x:leave_two_out_d(x))

### Leave 3 out

In [ ]:
# ## IMPORTANT: if leave-1-out/leave-2-out has already been run, you need to load users from the start

def leave_three_out_d(profile_items):
  dataset_list = []
  for item in profile_items:
      item_type = df_artifacts[df_artifacts['id'] == item]['type'].tolist()
      if item_type[0] == 'dataset':
          dataset_list.append(item)
          if len(dataset_list) == 3:
            for ds in dataset_list:
              profile_items.remove(ds)
            return dataset_list

In [ ]:
users_d['leave_three_out_d'] = users_d['id'].apply(lambda x:leave_three_out_d(x))

## Dataset Recommender

In [ ]:
users_d.reset_index(drop=True, inplace=True)

In [ ]:
authors = users_d.explode('id')

In [ ]:
users_g=nx.from_pandas_edgelist(authors, 'author_name', 'id')
G = nx.compose(giant, users_g)

In [ ]:
print(nx.info(G))

Name: 
Type: Graph
Number of nodes: 4059
Number of edges: 5119
Average degree:   2.5223


In [ ]:
g_emb = n2v(G, dimensions=128)

WINDOW = 4 # Node2Vec fit window
MIN_COUNT = 1 # Node2Vec min. count
BATCH_WORDS = 64 # Node2Vec batch words

mdl = g_emb.fit(
    window=WINDOW,
    min_count=MIN_COUNT,
    batch_words=BATCH_WORDS
)

# create embeddings dataframe
emb_df = (
    pd.DataFrame(
        [mdl.wv.get_vector(str(n)) for n in G.nodes()],
        index = G.nodes
    )
)

Computing transition probabilities:   0%|          | 0/4059 [00:00<?, ?it/s]

Generating walks (CPU: 1): 100%|██████████| 10/10 [02:45<00:00, 16.52s/it]


In [ ]:
dataset = df_artifacts.loc[(df_artifacts['type']=='dataset')]
dataset.shape

(567, 5)

In [ ]:
def predict_links(G, df, article_id, N):
    '''
    This function will predict the top N links a node (article_id) should be connected with
    which it is not already connected with in G.
    
    params:
        G (Netowrkx Graph) : The network used to create the embeddings
        df (DataFrame) : The dataframe which has embeddings associated to each node
        article_id (Integer) : The article you're interested 
        N (Integer) : The number of recommended links you want to return
        
    returns:
        This function will return a list of nodes the input node should be connected with.
    '''
    
    # separate target article with all others
    article = df[df.index == article_id]
    # other articles are all articles which the current doesn't have an edge connecting
    all_nodes = G.nodes()
    other_nodes = [n for n in all_nodes if n not in list(G.adj[article_id]) + [article_id]]

    arti_set = [item for item in other_nodes if item in dataset['id'].values] #Just recommend dataset 


    other_articles = df[df.index.isin(arti_set)]
    # get similarity of current reader and all other readers
    sim = cosine_similarity(article, other_articles)[0].tolist()
    idx = other_articles.index.tolist()
    # create a similarity dictionary for this user w.r.t all other users
    idx_sim = dict(zip(idx, sim))
    idx_sim = sorted(idx_sim.items(), key=lambda x: x[1], reverse=True)
    
    similar_articles = idx_sim[:N]
    articles = [art[0] for art in similar_articles]
    return articles
  
#predict_links(G = G, df = emb_df, article_id = "Goeree, Ron", N = 10)

In [ ]:
def recommend(name):
  recommended_list = predict_links(G = G, df = emb_df, article_id = name, N = 10)
  return recommended_list

In [ ]:
users_d['recommended_items'] = users_d['author_name'].apply(lambda x:recommend(x))

### KEYWORD EVALUATION

In [ ]:
def get_keys(profile):
  mylist = []
  for id_p in profile:
    mylist.append(df_artifacts[df_artifacts['id'] == id_p]['keyword'].tolist()[0])
  return mylist

In [ ]:
users_d['profile_keywords'] = users_d['id'].map(lambda x:get_keys(x))

In [ ]:
users_d['recommended_keywords'] = users_d['recommended_items'].map(lambda x:get_keys(x))

In [ ]:
users_d['profile_keywords'] = users_d['profile_keywords'].map(lambda x:list(set([item if type(sublist) is list else sublist for sublist in x for item in sublist])))

In [ ]:
users_d

,author_name,id,d_counter,recommended_items,profile_keywords,recommended_keywords
0,"A. Crone, Eveline",[50|r37980778c78::929a30378eb172e271714387c0d5...,4,[50|doi_dedup___::0320a9510932366b7b791160306d...,"[psychology, emotions, cem, cortex, Cognitive ...","[[Social Sciences, 2016, Brain, human activiti..."
1,"A. Jacobsson, Josefin",[50|r37980778c78::3733d784e60cb4a92ec8226b7564...,4,[50|dedup_wf_001::7dcdcd37da14303ab260ccbf6a1e...,"[volumes, nervous system, regression, highest,...","[[Medicine, Sociology, Biological Sciences, ne..."
2,"A. Norman, Kenneth",[50|r37980778c78::3dde9a8ae6db8d35cdfb4fd5585c...,6,[50|dedup_wf_001::bf8366fdeceffa41f9c1ce5fb20e...,"[bayesian, Probability theory, Functional magn...","[[Brain, Biology, Cell Biology, Neuroscience, ..."
3,"A. Turner, Jessica",[50|r37980778c78::c0dbf0428951817d1855dd0b2c5a...,6,[50|dedup_wf_001::3c097cea54f883d400794964d8a1...,"[Magnetic resonance imaging, Neuropsychiatric ...","[[Molecular Biology, Neuroscience, Physiology,..."
4,"Afzali, Maryam",[50|r37980778c78::6f1ef756cf61134a31340a05042f...,6,[50|r37980778c78::df43de3287fcb267ce569a387aff...,"[RTOP, Cell Biology, Biophysics, technique, AM...","[[Uncategorised, focus, fa, analysis, impact, ..."
...,...,...,...,...,...,...
148,"Yamada, Takashi",[50|doi_dedup___::6f9d880df4bfd4c4aaa2c2fe0f39...,4,[50|r37980778c78::5cc14d49f1f6302b4a3da4f161b9...,"[Abnormality, Schizophrenia, Algorithms, Neura...","[[Physics, Cell Biology, Neuroscience, Mathema..."
149,"Yang, Yang",[50|r37980778c78::6190c74f564a6118cad28ab5349e...,5,[50|dedup_wf_001::3d1c061bf7fd4ea8f6e97e8958c4...,"[Research and Analysis Methods, Medicine and H...","[[Uncategorised, right motor areas, ischemic s..."
150,"Yu, Chunshui",[50|r37980778c78::3360403652895b1d322d59a72e11...,8,[50|r37980778c78::8d84e4cededf28a6e1ae079981b9...,"[Schizophrenia, Internal medicine, Adult, Visu...","[[Medicine, Biological Sciences, neuroscience,..."
151,"Zhao, Zhilian",[50|r37980778c78::af00ac7d9359d7e4b018bf6887dc...,6,[50|r37980778c78::8d84e4cededf28a6e1ae079981b9...,"[Statistical parametric mapping, Case-Control ...","[[Medicine, Biological Sciences, neuroscience,..."


In [ ]:
def match_counter(user):
  count = 0
  mylist = []
  for item_list in user['recommended_keywords']:
    for item in item_list:
      if item in user["profile_keywords"]:
        count+=1
    mylist.append(count)
    count = 0
  return mylist

In [ ]:
users_d['kw_counter'] = users_d.apply(lambda row : match_counter(row),axis=1)

In [ ]:
def norm_counters(user):
  new_list = []
  for item in user['kw_counter']:
    new_item = item/len(user['profile_keywords'])
    new_list.append(new_item)
  return new_list

In [ ]:
users_d['norm_counter'] = users_d.apply(lambda x:norm_counters(x),axis=1)

In [ ]:
users_d

,author_name,id,d_counter,recommended_items,profile_keywords,recommended_keywords,kw_counter,norm_counter
0,"A. Crone, Eveline",[50|r37980778c78::929a30378eb172e271714387c0d5...,4,[50|doi_dedup___::0320a9510932366b7b791160306d...,"[psychology, emotions, cem, cortex, Cognitive ...","[[Social Sciences, 2016, Brain, human activiti...","[0, 2, 4, 0, 0, 0, 0, 0, 4, 2]","[0.0, 0.07407407407407407, 0.14814814814814814..."
1,"A. Jacobsson, Josefin",[50|r37980778c78::3733d784e60cb4a92ec8226b7564...,4,[50|dedup_wf_001::7dcdcd37da14303ab260ccbf6a1e...,"[volumes, nervous system, regression, highest,...","[[Medicine, Sociology, Biological Sciences, ne...","[7, 8, 4, 4, 4, 6, 10, 6, 3, 5]","[0.1590909090909091, 0.18181818181818182, 0.09..."
2,"A. Norman, Kenneth",[50|r37980778c78::3dde9a8ae6db8d35cdfb4fd5585c...,6,[50|dedup_wf_001::bf8366fdeceffa41f9c1ce5fb20e...,"[bayesian, Probability theory, Functional magn...","[[Brain, Biology, Cell Biology, Neuroscience, ...","[0, 1, 0, 0, 3, 3, 3, 0, 0, 0]","[0.0, 0.041666666666666664, 0.0, 0.0, 0.125, 0..."
3,"A. Turner, Jessica",[50|r37980778c78::c0dbf0428951817d1855dd0b2c5a...,6,[50|dedup_wf_001::3c097cea54f883d400794964d8a1...,"[Magnetic resonance imaging, Neuropsychiatric ...","[[Molecular Biology, Neuroscience, Physiology,...","[15, 6, 6, 6, 6, 2, 1, 11, 11, 11]","[0.7142857142857143, 0.2857142857142857, 0.285..."
4,"Afzali, Maryam",[50|r37980778c78::6f1ef756cf61134a31340a05042f...,6,[50|r37980778c78::df43de3287fcb267ce569a387aff...,"[RTOP, Cell Biology, Biophysics, technique, AM...","[[Uncategorised, focus, fa, analysis, impact, ...","[0, 0, 0, 0, 0, 4, 4, 4, 4, 4]","[0.0, 0.0, 0.0, 0.0, 0.0, 0.2222222222222222, ..."
...,...,...,...,...,...,...,...,...
148,"Yamada, Takashi",[50|doi_dedup___::6f9d880df4bfd4c4aaa2c2fe0f39...,4,[50|r37980778c78::5cc14d49f1f6302b4a3da4f161b9...,"[Abnormality, Schizophrenia, Algorithms, Neura...","[[Physics, Cell Biology, Neuroscience, Mathema...","[2, 3, 2, 1, 4, 4, 4, 1, 4, 3]","[0.028985507246376812, 0.043478260869565216, 0..."
149,"Yang, Yang",[50|r37980778c78::6190c74f564a6118cad28ab5349e...,5,[50|dedup_wf_001::3d1c061bf7fd4ea8f6e97e8958c4...,"[Research and Analysis Methods, Medicine and H...","[[Uncategorised, right motor areas, ischemic s...","[0, 2, 17, 4, 4, 1, 2, 2, 3, 2]","[0.0, 0.02127659574468085, 0.18085106382978725..."
150,"Yu, Chunshui",[50|r37980778c78::3360403652895b1d322d59a72e11...,8,[50|r37980778c78::8d84e4cededf28a6e1ae079981b9...,"[Schizophrenia, Internal medicine, Adult, Visu...","[[Medicine, Biological Sciences, neuroscience,...","[11, 11, 3, 2, 3, 2, 1, 3, 11, 9]","[0.10679611650485436, 0.10679611650485436, 0.0..."
151,"Zhao, Zhilian",[50|r37980778c78::af00ac7d9359d7e4b018bf6887dc...,6,[50|r37980778c78::8d84e4cededf28a6e1ae079981b9...,"[Statistical parametric mapping, Case-Control ...","[[Medicine, Biological Sciences, neuroscience,...","[10, 12, 6, 7, 5, 7, 6, 6, 8, 9]","[0.10526315789473684, 0.12631578947368421, 0.0..."


### Hit-rate@1 Evaluation

In [ ]:
def hit_rate_d(row):
  for item in row["recommended_items"]:
    if item == row['leave_one_out_d']:
      return 1
  return 0 

In [ ]:
users_d['hit_rate_d'] = users_d.apply(lambda x:hit_rate_d(x),axis=1)

In [ ]:
users_d['hit_rate_d'].mean()

0.9411764705882353

### Hit-rate@2 Evaluation

In [ ]:
def hit_rate_d(row):
  for item in row["recommended_items"]:
    if item in row['leave_two_out_d']:
      return 1
  return 0 

In [ ]:
users_d['hit_rate_d'] = users_d.apply(lambda x:hit_rate_d(x),axis=1)

In [ ]:
users_d['hit_rate_d'].mean()

0.9869281045751634

### Hit-rate@3 Evaluation

In [ ]:
users_d[users_d['leave_three_out_d'].isna()]

,author_name,id,d_counter,leave_three_out_d,recommended_items


In [ ]:
def hit_rate_d(row):
  for item in row["recommended_items"]:
    if item in row['leave_three_out_d']:
      return 1
  return 0 

In [ ]:
users_d['hit_rate_d'] = users_d.apply(lambda x:hit_rate_d(x),axis=1)

In [ ]:
users_d['hit_rate_d'].mean()

0.9934640522875817

## Preparing data for software RecSys

In [ ]:
users_sw = pd.read_json('/content/gdrive/MyDrive/Recommendation_Systems_Activities/Journal/Data/ni_3900/users_ni_3900.json',orient='index')

In [ ]:
users_sw = users_sw.loc[users_sw.astype(str).drop_duplicates(subset=['id']).index]

In [ ]:
def sw_counter(user_prof):
  sw_counter = 0
  for id in user_prof:
    item_type = df_artifacts[df_artifacts['id'] == id]['type']
    for i in item_type:
      if str(i) == 'publication':
        sw_counter += 1
  return sw_counter

In [ ]:
users_sw['p_counter'] = users_sw['id'].apply(lambda row : sw_counter(row))

In [ ]:
users_sw = users_sw[users_sw['sw_counter'] != 0 ]
users_sw = users_sw[users_sw['sw_counter'] != 1 ]
users_sw = users_sw[users_sw['sw_counter'] != 2 ]

### some tests on users

In [ ]:
users_sw['sw_counter'].value_counts()

49    3
72    3
13    2
15    2
17    2
16    1
7     1
32    1
8     1
25    1
3     1
11    1
64    1
50    1
26    1
12    1
58    1
35    1
52    1
24    1
Name: sw_counter, dtype: int64

In [ ]:
test = users_sw.explode('id')

In [ ]:
rep = test['id'].value_counts()

In [ ]:
rep.value_counts()

10    16
1     15
12     9
15     7
20     6
9      6
19     4
4      4
5      3
7      3
11     3
8      2
16     2
17     2
18     2
13     1
14     1
6      1
3      1
2      1
22     1
Name: id, dtype: int64

### Leave 1 out

In [ ]:
def leave_one_out_sw(profile_items):
   for item in profile_items:
      item_type = df_artifacts[df_artifacts['id'] == item]['type'].tolist()
      if item_type[0] == 'software':
        profile_items.remove(item)
        return item

In [ ]:
users_sw['leave_one_out_sw'] = users_sw['id'].apply(lambda x:leave_one_out_sw(x))

### Leave 2 out

In [ ]:
def leave_two_out_sw(profile_items):
  sw_list = []
  counter = 0
  for item in profile_items:
      item_type = df_artifacts[df_artifacts['id'] == item]['type'].tolist()
      if item_type[0] == 'software':
          sw_list.append(item)
          counter+=1
          profile_items.remove(item)
          if counter == 2:
            return sw_list

In [ ]:
users_sw['leave_two_out_sw'] = users_sw['id'].apply(lambda x:leave_two_out_sw(x))

### Leave 3 out

In [ ]:
def leave_three_out_sw(profile_items):
  sw_list = []
  counter = 0
  for item in profile_items:
      item_type = df_artifacts[df_artifacts['id'] == item]['type'].tolist()
      if item_type[0] == 'software':
          sw_list.append(item)
          counter+=1
          profile_items.remove(item)
          if counter == 3:
            return sw_list

In [ ]:
users_sw['leave_three_out_sw'] = users_sw['id'].apply(lambda x:leave_three_out_sw(x))

## Software Recommender

In [ ]:
users_sw.reset_index(drop=True, inplace=True)

In [ ]:
authors = users_sw.explode('id')

In [ ]:
users_g=nx.from_pandas_edgelist(authors, 'author_name', 'id')

In [ ]:
G = nx.compose(giant, users_g)

In [ ]:
print(nx.info(G))

Name: 
Type: Graph
Number of nodes: 3933
Number of edges: 4864
Average degree:   2.4734


In [ ]:
g_emb = n2v(G, dimensions=64)

WINDOW = 4 # Node2Vec fit window
MIN_COUNT = 1 # Node2Vec min. count
BATCH_WORDS = 64 # Node2Vec batch words

mdl = g_emb.fit(
    window=WINDOW,
    min_count=MIN_COUNT,
    batch_words=BATCH_WORDS
)

# create embeddings dataframe
emb_df = (
    pd.DataFrame(
        [mdl.wv.get_vector(str(n)) for n in G.nodes()],
        index = G.nodes
    )
)

emb_df.head()

Computing transition probabilities:   0%|          | 0/3933 [00:00<?, ?it/s]

Generating walks (CPU: 1): 100%|██████████| 10/10 [02:57<00:00, 17.78s/it]


,0,1,2,3,4,5,6,7,8,9,...,54,55,56,57,58,59,60,61,62,63
50|doi_dedup___::75a230ede9917ccb2b7aeb83a913348e,-0.335076,1.252077,0.214705,0.990315,0.008131,0.324026,0.277074,0.283510,0.078628,0.973506,...,0.060510,-0.627052,-1.238817,-0.500939,0.851494,-0.268488,0.796210,-0.506276,-0.426384,-0.108745
50|doi_dedup___::f9fa75ddaef9f0cab5ca7aeee6988d60,-0.251877,0.841525,0.030137,0.442109,-0.063258,-0.272283,0.055829,-0.400199,0.355207,-0.488140,...,0.230881,-0.527694,-0.258084,-0.429581,0.482505,0.402251,-0.427001,0.646252,0.057944,-0.205848
50|doi_dedup___::c98b296dcebb6595d8bb351bc0da7776,-0.301024,0.767318,0.074684,0.881000,-0.310112,0.466022,0.253582,-0.515656,0.531293,-0.149149,...,-0.419637,-0.745326,-0.048454,-0.000379,0.200701,0.454135,0.071742,0.236818,0.655824,-0.491339
50|doi_dedup___::afb5184f02eb2636845e3853e75215b9,-0.656441,0.631691,-0.054582,1.795555,1.086463,0.035012,-0.074797,0.308271,0.004586,0.344662,...,-0.317337,-1.451530,0.078382,-0.559890,0.305573,-0.290026,0.245945,-0.109634,0.353083,0.449351
50|doi_dedup___::70455afcfabc980e3e135bd64bb080bd,-1.554420,0.183332,0.211987,1.571259,0.399722,-1.454605,-0.667433,-0.584201,0.433066,0.290477,...,-0.250336,-0.543588,-1.435574,-0.283219,0.211554,-0.246418,-0.195388,-0.172123,0.754514,-0.188991


In [ ]:
software = df_artifacts.loc[(df_artifacts['type']=='software')]

In [ ]:
test_software = software.loc[software.astype(str).drop_duplicates(subset=['keyword']).index]

In [ ]:
test_software

,keyword,type,id,en_abstract,en_title
0,"[neuroimaging, workflow, pipeline, preprocessi...",software,50|doi_________::77d25c5587f0b1ab44b34485d4ecb37f,Hotfix Release <ul> <li>[FIX] Breadcrumb repor...,poldracklab/fmriprep: 1.2.5
4,"[neuroimaging, workflow, pipeline, statistics,...",software,50|doi_________::b72d03a0b3d6a87011d520606135f540,Release Notes <p>New feature release in the 0....,poldracklab/fitlins: 0.7.0
20,[],software,50|doi_________::33d2146ba969a6368920d0f4819d8dc2,<p>CHANGES</p> <p>A first release that gives T...,TemplateFlow: standardized neuroimaging standa...
1918,"[neuroimaging, workflow, pipeline, preprocessi...",software,50|doi_dedup___::2f2b564c5055a3b818dc92d69536b9b0,<p>Hotfix release to retrieve correct version ...,TemplateFlow Client: Accessing the library of ...
1919,"[Cell Biology, Neuroscience, Physiology, Evolu...",software,50|doi_dedup___::d1e4df32d1acb2c2b81455857c8bb3fb,Preprocessing of functional MRI (fMRI) involve...,fMRIPrep: a robust preprocessing pipeline for ...
1921,"[neuroimaging, workflow, pipeline, preprocessi...",software,50|doi_dedup___::814fb54a5f9572c5dbacf953f6fa6ade,"<p>A collection of Nipype interfaces, patches ...",NIWorkflows: NeuroImaging Workflows
1922,"[neuroimaging, workflow, pipeline, preprocessi...",software,50|doi_dedup___::3d139b81b30e2c151375fd9da4ad48ed,<p>dMRIPrep is a robust and easy-to-use pipeli...,dMRIPrep: a robust preprocessing pipeline for ...
1926,"[neuroimaging, workflow, pipeline, preprocessi...",software,50|doi_dedup___::cb1d46ce16547c0a1e191ad4a9ef6d52,"<p>A BIDS-App to preprocess anatomical (T1w, T...",sMRIPrep: Structural MRI PREProcessing workflows
1939,"[neuroimaging, workflow, pipeline, preprocessi...",software,50|doi_dedup___::528fe145915ceda3e385c1907fb25a7b,"Release Notes <p>Improves the documentation, w...",Poldracklab/Fmriprep Untagged-Aad77D0215Efb289...
1956,"[neuroimaging, workflow, pipeline, preprocessi...",software,50|doi_dedup___::65b5a171d46983aaf71588d8cb8c4c94,Release Notes <p>Hotfix release intended for D...,poldracklab/fmriprep: 1.3.0.post1


In [ ]:
software.en_title.str.count("Fmriprep|fmriprep").sum()

48

In [ ]:
def predict_links(G, df, article_id, N):
    '''
    This function will predict the top N links a node (article_id) should be connected with
    which it is not already connected with in G.
    
    params:
        G (Netowrkx Graph) : The network used to create the embeddings
        df (DataFrame) : The dataframe which has embeddings associated to each node
        article_id (Integer) : The article you're interested 
        N (Integer) : The number of recommended links you want to return
        
    returns:
        This function will return a list of nodes the input node should be connected with.
    '''
    
    # separate target article with all others
    article = df[df.index == article_id]
    
    # other articles are all articles which the current doesn't have an edge connecting
    all_nodes = G.nodes()
    other_nodes = [n for n in all_nodes if n not in list(G.adj[article_id]) + [article_id]]

    arti_set = [item for item in other_nodes if item in software['id'].values] #Just recommend SW 


    other_articles = df[df.index.isin(arti_set)]
    
    # get similarity of current reader and all other readers
    sim = cosine_similarity(article, other_articles)[0].tolist()
    idx = other_articles.index.tolist()
    
    # create a similarity dictionary for this user w.r.t all other users
    idx_sim = dict(zip(idx, sim))
    idx_sim = sorted(idx_sim.items(), key=lambda x: x[1], reverse=True)
    
    similar_articles = idx_sim[:N]
    articles = [art[0] for art in similar_articles]
    return articles
  
#predict_links(G = G, df = emb_df, article_id = "Goeree, Ron", N = 10)

In [ ]:
def recommend(name):
  recommended_list = predict_links(G = G, df = emb_df, article_id = name, N = 10)
  return recommended_list  

In [ ]:
users_sw['recommended_items'] = users_sw['author_name'].apply(lambda x:recommend(x))

### Software RecSys Evaluation

In [ ]:
def hit_rate_sw(row):
  for item in row["recommended_items"]:
    if item in row['leave_one_out_sw']:
      return 1
  return 0 

### Hit-Rate@3 

In [ ]:
users_sw['id'][16]

['50|doi_________::7d676e077d456dcf641976498eb6c693']

In [ ]:
users_sw[users_sw['leave_three_out_sw'].isna()]

,author_name,id,sw_counter,leave_three_out_sw,recommended_items
16,"Lee, John A.",[50|doi_________::7d676e077d456dcf641976498eb6...,3,None,[50|doi_________::7583b21864e2343d9c0dec0dab4b...


In [ ]:
def hit_rate_sw(row):
  print(row['recommended_items'])
  for item in row["recommended_items"]:
    if item in row['leave_three_out_sw']:
      return 1
  return 0 

In [ ]:
users_sw['hit_rate_sw'] = users_sw.apply(lambda x:hit_rate_sw(x),axis=1)

['50|doi_________::77d25c5587f0b1ab44b34485d4ecb37f', '50|doi_________::83fcc5aeb29150baf8888487dd2eefb4', '50|doi_________::a02d76649a652bc7f601eccae5b5d395', '50|doi_________::2fcf8da2801587a325c42e570badc916', '50|doi_________::cb1d5119cad326b4dcacd8fad2ff4421', '50|doi_________::3987780588ec254078aad955fb4186b4', '50|doi_________::3bdc580b11ad594f079d62ae17459c77', '50|doi_________::cd68d9491b2528afdb55325b17b9f4e2', '50|doi_________::a0bb81aaaac49b6415c7a58d2645520c', '50|doi_dedup___::528fe145915ceda3e385c1907fb25a7b']
['50|doi_________::77d25c5587f0b1ab44b34485d4ecb37f', '50|doi_dedup___::3d819fd8fcecd35cdbbf271ba97c737d', '50|doi_________::c1881ca8737b1dd96123c03e5f11e4c2', '50|doi_________::fe8f2e909e6e8e6e3b8e4726f23657bb', '50|doi_________::12ff2c661e87ba8d7c4fc6eb9053aef4', '50|doi_________::33d2146ba969a6368920d0f4819d8dc2', '50|doi_________::3cb002309a835e9f368e1f339f7c988c', '50|doi_________::3d426b8bb216010bc493e8e57e3a2e40', '50|doi_dedup___::0bb1987c461f348a3aa43f6ce0

TypeError: ignored

In [ ]:
users_sw['hit_rate_sw'].mean()

0.8518518518518519

## Preparing data for Combined RecSys

In [ ]:
users = pd.read_json('/content/gdrive/MyDrive/Recommendation_Systems_Activities/Journal/Data/users_june21/users_v3.json')

In [ ]:
#users_last.columns = ['user','profile']
#for i,x in enumerate(users_last['profile']):
 # users_last['profile'][i] = users_last['profile'][i]['data']

In [ ]:
# def get_user(user_list):
#   mylist = []
#   for item in user_list:
#     for items in nodes_list:
#       if item == items:
#         mylist.append(item)
#   return mylist

In [ ]:
# users['profile_items'] = users['profile'].apply(lambda x:get_user(x))

In [ ]:
# cln_users = users[users['profile_items'].map(len) > 6]

In [ ]:
def clean_users(user_prof):
  pub_counter = 0
  sd_counter = 0
  for id in user_prof:
    item_type = df_artifacts[df_artifacts['id'] == id]['type']
    for i in item_type:
      if str(i) == 'publication':
        pub_counter += 1
      else:
        sd_counter += 1
  return pub_counter

In [ ]:
users['pub_counter'] = users['id'].apply(lambda row : clean_users(row))

In [ ]:
users

,author_name,id,sd_counter,pub_counter
0,"A. Norman, Kenneth",[50|r37980778c78::3dde9a8ae6db8d35cdfb4fd5585c...,6,0
1,"A. Turner, Jessica",[50|r37980778c78::eca395a306186f95e5fbe3b16f68...,6,0
2,Adeel Razi,[50|doi_dedup___::796fadcb23853b089b8a1e644e7d...,0,6
3,"Aja-Fernández, Santiago",[50|r37980778c78::b52f346497320489e3de542b6bdd...,9,1
4,Alzheimer’s Disease Neuroimaging Initiative,[50|doi_dedup___::3cbda8b2577a3b5dc90ba520339c...,0,6
...,...,...,...,...
351,"Zhao, Zhilian",[50|r37980778c78::af00ac7d9359d7e4b018bf6887dc...,6,1
352,"Zhong, Ning",[50|r37980778c78::4fcd43dfa35532d0819423b5b7c4...,5,2
353,"Zhou, Haiyan",[50|r37980778c78::4fcd43dfa35532d0819423b5b7c4...,5,1
354,"de Luis-García, Rodrigo",[50|r37980778c78::b52f346497320489e3de542b6bdd...,9,1


In [ ]:
import random

In [ ]:
def leave_one_cv(profile_items):
   for item in profile_items:
      item_type = df_artifacts[df_artifacts['id'] == item]['type'].tolist()
      if item_type[0] != 'publication':
        users['id'] = users['id'].apply(lambda x:[i for i in x if i != item])
        return item
      

In [ ]:
users['leave_one_cv'] = users['id'].apply(lambda x:leave_one_cv(x))

In [ ]:
users

,author_name,id,sd_counter,pub_counter,leave_one_cv
0,"A. Norman, Kenneth",[50|r37980778c78::f438c4e29b0015775392812d048b...,6,0,50|r37980778c78::3dde9a8ae6db8d35cdfb4fd5585c1dd8
1,"A. Turner, Jessica",[50|r37980778c78::ecde2547d145a2c4583f0418616a...,6,0,50|r37980778c78::eca395a306186f95e5fbe3b16f685737
2,"Aja-Fernández, Santiago",[50|r37980778c78::8bb673ac2e78ae84d6fce829dd69...,9,1,50|r37980778c78::b52f346497320489e3de542b6bdd9a3f
3,"Amirbekian, Bago",[50|doi_________::28b4308a6ed463db39159e7a2038...,14,0,50|doi_________::c366b2de1b4e34c37362d5eb8f43052d
4,"Amlien, Inge K.",[50|doi_________::77d25c5587f0b1ab44b34485d4ec...,10,1,50|doi_dedup___::d1e4df32d1acb2c2b81455857c8bb3fb
...,...,...,...,...,...
298,"Zhao, Zhilian",[50|r37980778c78::c6743c483b6d1cbdf84bc978e2e3...,6,1,50|r37980778c78::af00ac7d9359d7e4b018bf6887dc978a
299,"Zhong, Ning",[50|r37980778c78::d7cd259ffdca8d25fe082d779c79...,5,2,50|r37980778c78::4fcd43dfa35532d0819423b5b7c47208
300,"Zhou, Haiyan",[50|r37980778c78::d7cd259ffdca8d25fe082d779c79...,5,1,50|r37980778c78::4fcd43dfa35532d0819423b5b7c47208
301,"de Luis-García, Rodrigo",[50|r37980778c78::8bb673ac2e78ae84d6fce829dd69...,9,1,50|r37980778c78::b52f346497320489e3de542b6bdd9a3f


In [ ]:
users.reset_index(drop=True, inplace=True)

In [ ]:
authors = users.explode('id')

## Dataset and Software Combined Recommender

In [ ]:
# Build your graph
users_g=nx.from_pandas_edgelist(authors, 'author_name', 'id')
G = nx.compose(giant, users_g)

In [ ]:
print(nx.info(G))

Name: 
Type: Graph
Number of nodes: 2688
Number of edges: 5699
Average degree:   4.2403


In [ ]:
from node2vec import Node2Vec as n2v

In [ ]:
g_emb = n2v(G, dimensions=32)

WINDOW = 1 # Node2Vec fit window
MIN_COUNT = 1 # Node2Vec min. count
BATCH_WORDS = 4 # Node2Vec batch words

mdl = g_emb.fit(
    window=WINDOW,
    min_count=MIN_COUNT,
    batch_words=BATCH_WORDS
)

# create embeddings dataframe
emb_df = (
    pd.DataFrame(
        [mdl.wv.get_vector(str(n)) for n in G.nodes()],
        index = G.nodes
    )
)

emb_df.head()

Computing transition probabilities:   0%|          | 0/2688 [00:00<?, ?it/s]

Generating walks (CPU: 1): 100%|██████████| 10/10 [01:46<00:00, 10.61s/it]


,0,1,2,3,4,5,6,7,8,9,...,22,23,24,25,26,27,28,29,30,31
50|doi_dedup___::c760ec69bacc7756290f5620878e1d8c,0.029763,0.606829,0.048690,-0.980313,-0.014839,0.395438,-0.581391,0.250080,0.553973,1.523950,...,-0.107582,0.236983,1.015973,0.296526,-0.024863,-0.489872,0.590836,0.018095,-0.857422,1.129327
50|r37980778c78::01e0b5345a2c98d13eab157e589fcf52,0.337239,-0.404531,-0.557752,-1.031410,0.025945,-0.461189,0.221554,-0.232739,0.988201,0.963006,...,0.028590,-1.213983,1.066908,-2.111471,0.666075,-0.384126,-1.563455,1.174580,-0.829265,1.603193
50|doi_dedup___::6f37c6c81214dac4bba084e01b176311,-0.118085,0.307497,-0.206834,-1.045097,0.811564,0.567154,-0.614397,0.324727,0.894610,0.392757,...,-0.619190,-0.246421,0.754153,0.302121,0.296183,-0.538999,0.740552,0.757182,0.283339,1.142517
50|doi_dedup___::3bbc46aa0a894c9451fa126f6ff1204a,-0.475310,0.607669,-0.179707,-0.611857,0.052649,0.822942,-0.183508,1.458391,1.008725,0.186517,...,-0.290684,-0.431252,0.669556,0.177260,0.345314,-0.316546,1.028838,0.612723,-0.376466,0.847404
50|doi_dedup___::0af0245adf676daa3ff28044472bc6e0,-1.254424,0.182702,-0.420853,0.280404,0.475492,0.370978,0.131690,1.256139,1.621113,1.981099,...,0.151103,0.178524,1.793900,-1.481936,0.051243,-1.033230,1.505823,1.083235,-0.544396,0.863776


###

In [ ]:
#df_artifacts = pd.read_json("/content/gdrive/MyDrive/Recommendation_Systems_Activities/TPDL2022/Data/data-with-subjects.json",lines=True)

ds = df_artifacts.loc[(df_artifacts['type']=='software') | (df_artifacts['type']=='dataset')]

In [ ]:
ds.shape

(604, 5)

In [ ]:
def predict_links(G, df, article_id, N):
    '''
    This function will predict the top N links a node (article_id) should be connected with
    which it is not already connected with in G.
    
    params:
        G (Netowrkx Graph) : The network used to create the embeddings
        df (DataFrame) : The dataframe which has embeddings associated to each node
        article_id (Integer) : The article you're interested 
        N (Integer) : The number of recommended links you want to return
        
    returns:
        This function will return a list of nodes the input node should be connected with.
    '''
    
    # separate target article with all others
    article = df[df.index == article_id]
    
    # other articles are all articles which the current doesn't have an edge connecting
    all_nodes = G.nodes()
    other_nodes = [n for n in all_nodes if n not in list(G.adj[article_id]) + [article_id]]

    arti_set = [item for item in other_nodes if item in ds['id'].values] #Just recommend dataset and software


    other_articles = df[df.index.isin(arti_set)]
    
    # get similarity of current reader and all other readers
    sim = cosine_similarity(article, other_articles)[0].tolist()
    idx = other_articles.index.tolist()
    
    # create a similarity dictionary for this user w.r.t all other users
    idx_sim = dict(zip(idx, sim))
    idx_sim = sorted(idx_sim.items(), key=lambda x: x[1], reverse=True)
    
    similar_articles = idx_sim[:N]
    articles = [art[0] for art in similar_articles]
    return articles
  
predict_links(G = G, df = emb_df, article_id = "Goeree, Ron", N = 10)

['50|r37980778c78::5ba3e79c05baeb92fcd3d7c9eed664e6',
 '50|r37980778c78::beb3f6cbefdf0675bb7df6b9f25ad184',
 '50|r37980778c78::ed97fc726065cb067f9c9dbdf3ab5672',
 '50|r37980778c78::eb307a1db4f6e41371ae65d837130e32',
 '50|r37980778c78::cf8f5a775337b4c002ef06362a719c77',
 '50|r37980778c78::178ccb0179a92a8f3ca77bc8d9dd47a3',
 '50|r37980778c78::b7514b2b1bfaf3919cb6c32a5d922ed4',
 '50|r37980778c78::a55e7ff471e4a2d8206daea5df35500d',
 '50|doi_dedup___::d96db72cb4922035721bb4bd3d6834f5',
 '50|doi_dedup___::d7419ac27ce85965a38a51281258c6a6']

In [ ]:
def recommend(name):
  recommended_list = predict_links(G = G, df = emb_df, article_id = name, N = 10)
  return recommended_list
    

In [ ]:
users['recommended_items'] = users['author_name'].apply(lambda x:recommend(x))

### Combined RecSys Evaluation

In [ ]:
def hit_rate(row):
  for item in row["recommended_items"]:
    if item == row['leave_one_cv']:
      return 1
  return 0 

In [ ]:
users['hit_rate'] = users.apply(lambda x:hit_rate(x),axis=1)

In [ ]:
pd.set_option('display.max_rows', None)


In [ ]:
users['hit_rate'].mean()

0.6138613861386139